## This notebook demonstrates how to propagate an orbit back in time, then compute a state transition matrix on the forward propagation

In [3]:
import adam
import sys, os
import datetime
import numpy as np
import numpy.testing as npt
import pandas as pd

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a config, see config_demo to get one.

In [5]:
cm = adam.ConfigManager()
cm.set_default_env('dev')

service = adam.Service.from_config(cm.get_config())
service.setup()

[0.574937] Setup


True

In [6]:
working_project = service.new_working_project()

Set up project with uuid 7ee19d92-f6f5-4e2c-a56c-468e568e8870
[1.383377] Create a new working project under project de77f430-7a4f-407c-93c1-a99cc4cd2909


### Definitions

Following are definitions of all the parameters we would like to use, including regarding the asteroid, its location, and the times we want to use in our propagations.

In [7]:
asteroid_params = {
    'mass': 500.5,              # object mass
    'solar_rad_area': 25.2,     # object solar radiation area (m^2)
    'solar_rad_coeff': 1.2,     # object solar radiation coefficient
    'drag_area': 33.3,          # object drag area (m^2)
    'drag_coeff': 2.5,          # object drag coefficient
    'object_name': 'KillerAsteroid'
}

initial_state_vector = [1.9614856544727500000e7,
                        1.4843205548512800000e8,
                        5.5892860024181600000e7,
                        -28.4909386978969000000,
                        -5.3762767683990300000,
                        2.5848941723961300000]

propagation_time = 50 * 365 * 24 * 60 * 60  # 50 years in seconds.

now = datetime.datetime(2018, 2, 21, 0, 0, 0, 123456)
fifty_years_ago = now - datetime.timedelta(seconds=propagation_time)

### Backwards propagation

This creates and runs a propagation backwards 50 years from now.

In [8]:
backwards_propagation = adam.Batch(
    adam.PropagationParams(
        {
            'start_time': now.isoformat() + 'Z',
            'end_time': fifty_years_ago.isoformat() + 'Z',
            'step_size': 0,  # Use adaptive step size.
            'project_uuid': working_project.get_uuid(),
        }
    ), adam.OpmParams(
        dict(
            asteroid_params, **{
            'epoch': now.isoformat() + 'Z',
            'state_vector': initial_state_vector,
        })
    )
)

batch_run_manager = adam.BatchRunManager(service.get_batches_module(), [backwards_propagation])
batch_run_manager.run();

# Should be 'COMPLETED'.
print('Finished backwards propagation in state %s' % (backwards_propagation.get_calc_state()))
state_vector_fifty_years_ago = backwards_propagation.get_results().get_end_state_vector()
print('State vector fifty years ago: %s' % state_vector_fifty_years_ago)

[0.975908] Submitting 1 runs.
[17.057464] Running.
[2.918979] Retrieving propagation results.
Finished backwards propagation in state COMPLETED
State vector fifty years ago: [-105079893.82460001, 55761751.36629, 39539057.706580006, -14.97306425914, -31.30823875332, -9.959259069193001]


TODO(laura): This is actually not true because python notebooks do not run cells simultaneously. There are ways to make this work. Do it.

In case you're curious, you can run the following to get the state of your runs (in this case only backwards_propagation) while running.

In [9]:
status = batch_run_manager.get_latest_statuses()
status_counts = [[s, len(status[s])] for s in status]
df = pd.DataFrame(status_counts, columns=['Task', 'Status'])
df.style.hide_index()

Task,Status
PENDING,0
RUNNING,0
COMPLETED,1
FAILED,0


### STM calculation

Now that we know where this asteroid was 50 years ago, let's propagate forward using the STM module to compute a state transition matrix that we can use in our delta-V calculations.

In [10]:
forwards_propagation_params = adam.PropagationParams(
    {
        'start_time': fifty_years_ago.isoformat() + 'Z',
        'end_time': now.isoformat() + 'Z',
        'step_size': 0,  # Use adaptive step size.
        'project_uuid': working_project.get_uuid(),
    }
)

forwards_opm_params = adam.OpmParams(
    dict(asteroid_params, **{
    'epoch': fifty_years_ago.isoformat() + 'Z',
    'state_vector': state_vector_fifty_years_ago,  # This will be nudged to compute the STM.
    })
)

stm_propagator = adam.StmPropagationModule(service.get_batches_module())
now_state_vector, stm = stm_propagator.run_stm_propagation(forwards_propagation_params, forwards_opm_params)

# We should have ended up quite close to the same position and velocity at which it started. Validate that.
difference = np.subtract(now_state_vector, initial_state_vector)
npt.assert_allclose(difference[0:3], [0, 0, 0], rtol=0, atol=1e-3)    # Allow 1m of change in position
npt.assert_allclose(difference[3:6], [0, 0, 0], rtol=0, atol=1e-10)   # But only a tiny change in velocity

# Now we have the state transition matrix!
print("State transition matrix:")
print(stm)

[0.76482] Submitting 13 runs.
[53.442117] Running.
[14.670358] Retrieving propagation results.


AssertionError: 
Not equal to tolerance rtol=0, atol=0.001

Mismatched elements: 3 / 3 (100%)
Max absolute difference: 2678752.6002275
Max relative difference: inf
 x: array([-2678752.600228,  -650883.233728,   184046.432238])
 y: array([0, 0, 0])

### Now what?

Now that we have the state transition matrix, how do we compute delta-Vs? What should happen now? TODO(hackathon)

In [11]:
# Cleans up working project and any batches and runs that were made within that project.
service.teardown()

Cleaning up working project 7ee19d92-f6f5-4e2c-a56c-468e568e8870...
[1.852966] Teardown
